In [ ]:
import time
import pip
def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])

In [ ]:
import_or_install('pymorphy2')
!pip install ruwordnet
!ruwordnet download

downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.2/ruwordnet.db


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import sys
import csv
from google.colab import files

from bs4 import BeautifulSoup

import nltk
import pymorphy2
from ruwordnet import RuWordNet

from tqdm import tqdm, notebook
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords as nltk_stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
useless_columns = ['AltName', 'Priority', 'Keyword', 'Article', 'Video', 'InfoManager', 'PriceMinimum', 'Image', 'VariantImg', 'Slider', 'SlideName', 'File', 'FileName', 
                   'ColorImages', 'ColorImagesNames', 'Provider', 'Functions', 'Upholstery', 'Material', 'Country', 'IsMain', 'IsNew', 'IsColor', 'isUcenka', 'BigImage', 'AltH1', 'SizeName', 
                   'Size', 'Tag', 'Type', 'Age', 'TypeBed', 'MaterialBed', 'IsComplect', 'Element', 'IsNotSeparately', 'IsRigit', 'IsWeek', 'IsNextWeek', 
                   'IsPriority', 'IsHideDescription', 'ReviewsCount', 'MinCount', 'PriorityPromo', 'FaqCount', 'DateStockUpdated', 'Stocks', 'SaleAuto', 'SaleAutoGroup', 'IsPriceUpdate', 
                   'Features', 'ImgPng', 'IsSaleIndex', 'DataPriceUpdate', 'ItemSizeGroup', 'ProductCondition', 'PrichinaUcenki', 'Toruda_Prices', 'IsBetonirovanie', 'RelatedGoods', 
                   'IsBetonirovanie_2', 'ReviewsAverage', 'NacenkaDostavkaUstanovka', 'StockUnitsFilter', 'TextSaleMarker', 'IsIndexNow', 'Description']

important_columns = ['ItemID', 'Message_ID', 'Parent_Message_ID', 'Details', 'Category', 'VariantTitle', 'VariantName', 'Seria', 'Model', 'Vendor', 'Sub_ID']

def download_file(dataframe, filename):
    dataframe.to_excel(filename + '.xlsx')
    files.download(filename + '.xlsx')

In [ ]:
random_state = 12345
csv.field_size_limit(sys.maxsize)

full_path = '/content/export-562-11-11-2021.csv'
text_file = open(full_path, 'r', encoding='utf-8-sig', errors='replace').read()
decoded_text_file = open(full_path, 'w', encoding='utf-8').write(text_file)
decoded_text = open(full_path, 'r', encoding='utf-8').read()
open('/content/Decoded_File.csv', 'w', encoding='1251', errors='replace').write(decoded_text)

37350338

In [ ]:
full_path = '/content/Decoded_File.csv'
df = pd.read_csv(full_path, encoding='1251', delimiter=';', decimal=',', engine = 'python', index_col=False)
df = df.rename(columns={'Name': 'Category'})

In [ ]:
df = df.drop_duplicates(subset=['ItemID'])

In [ ]:
columns_to_drop = []
for column in df.columns:
    if column not in important_columns:
        if 'Unnamed' in column:
            columns_to_drop.append(column)
        elif df[column].isna().sum() / len(df) > 0.5 or df[column].isna().sum() == len(df):
            columns_to_drop.append(column)
df = df.drop(columns_to_drop, axis=1)

In [ ]:
df = df.loc[df['InfoProductM'].isna() == False]

In [ ]:
def get_features_from_infomanager(text, re_expression):
    new_text = re.findall(re_expression, text)
    new_text = ''.join(new_text)
    return new_text

df['Volume'] = df['InfoProductM'].apply(get_features_from_infomanager, re_expression=('Объем: (.*?)Вес')).fillna(0)
df['PriceOPT'] = df['InfoProductM'].apply(get_features_from_infomanager, re_expression=('Закуп: (.*?)Поставщик')).fillna(0)
df['Vendor'] = df['InfoProductM'].apply(get_features_from_infomanager, re_expression=('Поставщик: (.*?)Наименование')).fillna('Unknown')
df['Weight'] = df['InfoProductM'].apply(get_features_from_infomanager, re_expression=('Вес: (.*?)Штук в упаковке')).fillna(0)

In [ ]:
df = df.drop(['InfoProductM'], axis=1)

In [ ]:
cols_to_numeric = ['Message_ID', 'Parent_Message_ID', 'Noindex', 'Volume', 'PriceOPT', 'Weight']
for column in cols_to_numeric:
    if column in df.columns:
        if column == 'PriceOPT' or column == 'Weight' or column == 'Volume':
            df[column] = df[column].str.replace(',', '.')
            df[column] = pd.to_numeric(df[column], errors='coerce')
            df[column] = df[column].fillna(0)
        else:
            df[column] = pd.to_numeric(df[column], errors='coerce')

In [ ]:
df = df.loc[df['PriceOPT'] > 0]

In [ ]:
df['IsParent'] = np.where((df['Parent_Message_ID'] == 0), 1, 0)

In [ ]:
df = df.loc[df['IsParent'] == 1]

In [ ]:
df['Noindex'] = df['Noindex'].fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df = df.loc[df['Noindex'] == 0]

In [ ]:
for column in df.columns:
    if column in useless_columns:
        df = df.drop(column, axis=1)

In [ ]:
def get_main_subid(text):
    new_text = text.split('.')
    new_text = new_text[0]
    return new_text

df['Main_ID_Razdel'] = df['Sub_ID'].apply(get_main_subid)

In [ ]:
df['FullName'] = df[['Category', 'VariantTitle', 'VariantName', 'Seria', 'Model']].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
df = df.drop(['VariantTitle', 'VariantName', 'Model'], axis=1)

In [ ]:
def clean(text):
    
    '''
    - Приводит текстовую строку к нижнему регистру,
    - Очищает от пробелов и других лишних символов
    - ???
    - Заменяет двойные пробелы единичными
    - Избавляется от скобок
    - Очищает объем внутри скобок, например (1400x900x300) заменяется на ()
    - ???
    - ???
    - Удаляет английские буквы
    '''
    
    if pd.isna(text) == True:
        return text
    else:
        text = text.lower()    
        text = re.sub(r"(?:\n|\r)", " ", text)
        text = re.sub(r"[^a-zA-Zа-яА-Я0-9 ]+", " ", text).strip()
        text = text.replace("  ", " ")
        text = re.sub(r'[0-9]{2,4}.[0-9]{2,4}.[0-9]{2,4}', '', text)
        text = re.sub('[\(\)]', '', text)
        text = re.sub('<[^<]+?>', '', text)
        return text

In [ ]:
def clean_numbers(text):
    text = re.sub('[0-9]', "", str(text))
    text = text.replace("  ", " ")
    return text

In [ ]:
def clean_html(text):
    '''
    Очищает текстовую строку от HTML кода с помощью BeatifulSoup
    '''
    
    if pd.isna(text) == True:
        return text
    else:
        soup = BeautifulSoup(text)
        text = soup.get_text(" ")
        return text

In [ ]:
df['Details'] = df['Details'].apply(clean_html)
df['Details'] = df['Details'].apply(clean_numbers)
df['Details'] = df['Details'].apply(clean)
df['Details'] = df['Details'].replace(r'\s+',' ',regex=True)
df = df.loc[df['Details'].isna() == False]

In [ ]:
df['Seria'] = df['Seria'].fillna('Отсутствует')

In [ ]:
stop_words = set(nltk_stopwords.words('russian'))
morph = MorphAnalyzer()

Loading dictionaries from /usr/local/lib/python3.7/dist-packages/pymorphy2_dicts_ru/data
format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [ ]:
%%time
def lemmatize(doc):
    tokens = []
    for token in doc.split():
        if token and token not in stop_words:
            token = token.strip()
            token = morph.parse(token)[0]
            token = token.normal_form
            tokens.append(token)
    if len(tokens) > 0:
        return " ".join(tokens)
    return None
df['Details'] = df['Details'].apply(lemmatize)

CPU times: user 22.6 s, sys: 26.3 ms, total: 22.6 s
Wall time: 22.7 s


In [ ]:
df['Details'] = df['Details'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['Details'] = df['Details'].str.strip().str.split(' ')

In [ ]:
additional_stopwords = ['характеристика', 'цвет', 'габарит', 'размер', 'который', 'это', 'вес', 'шхвхг', 
                        'вхшхг', 'прекрасно', 'широта', 'также', 'ваш', 'менеджер', 'уточнять', 'квт']
df['Details'] = df['Details'].apply(lambda x: ([word for word in x if word not in (additional_stopwords)]))

In [ ]:
from collections import Counter
def new_lemmatize(lemmatize_list):
    cleared_lemmatize_list = []
    for i in lemmatize_list:
        i = re.sub('[a-zA-Z]', '', i).strip()
        if len(i) > 3 and i != " ":
            cleared_lemmatize_list.append(i)
    return cleared_lemmatize_list

df['Details'] = df['Details'].apply(new_lemmatize)
df = df.loc[df.astype(str)['Details'] != '[]']

In [ ]:
def predict_n_clusters(X, clusters_range, x_type=None):
    sil_score_max = -1
    
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)

    for i in range(2,clusters_range + 1):
        model = KMeans(n_clusters=i, init='k-means++', n_init=5, max_iter=1000, tol=0.000001, random_state=random_state)
        labels = model.fit_predict(X)
        sil_score = silhouette_score(X, labels)
        print("The average silhouette score for %i clusters is %0.2f" %(i,sil_score))
        if sil_score > sil_score_max:
            sil_score_max = sil_score
            best_n_clusters = i
            if round(sil_score, 2) == 1.00:
                break
def clusterization_kmeans(X, n_clusters, x_type=None):
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)

    model = KMeans(n_clusters=n_clusters, init='k-means++', n_init=5, max_iter=1000, tol=0.000001, random_state=random_state)
    model.fit(X)
    clusters = model.predict(X)
    return clusters

In [ ]:
predict_n_clusters(df[['Volume', 'Weight']], 20)

The average silhouette score for 2 clusters is 0.52
The average silhouette score for 3 clusters is 0.44
The average silhouette score for 4 clusters is 0.54
The average silhouette score for 5 clusters is 0.51
The average silhouette score for 6 clusters is 0.51
The average silhouette score for 7 clusters is 0.49
The average silhouette score for 8 clusters is 0.49
The average silhouette score for 9 clusters is 0.45
The average silhouette score for 10 clusters is 0.46
The average silhouette score for 11 clusters is 0.45
The average silhouette score for 12 clusters is 0.46
The average silhouette score for 13 clusters is 0.44
The average silhouette score for 14 clusters is 0.45
The average silhouette score for 15 clusters is 0.45
The average silhouette score for 16 clusters is 0.45
The average silhouette score for 17 clusters is 0.46
The average silhouette score for 18 clusters is 0.46
The average silhouette score for 19 clusters is 0.45
The average silhouette score for 20 clusters is 0.45


In [ ]:
df['SizeCluster'] = clusterization_kmeans(df[['Volume', 'Weight']], 4)

In [ ]:
df['PriceCluster'] = pd.qcut(df['PriceOPT'], q=10, labels=range(0, 10))

In [ ]:
df = df.reset_index()

In [ ]:
def create_diff_matrix(col):
    x = df[col].to_numpy().reshape(-1, 1)
    y = df[col].to_numpy().reshape(1, -1)
    diff_matrix = np.zeros(shape=(len(df)+1,len(df)+1))
    diff_matrix = x - y
    return diff_matrix

In [ ]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=stop_words)

details_corpus = df['Details'].astype('U').values
details_matrix = tf.fit_transform(details_corpus)

coeff_list = [0.55, 0.125, 0.125, 0.2]

if sum(coeff_list) == 1.0:
    cosine_similarities_details = cosine_similarity(details_matrix, details_matrix) * coeff_list[0]
    cosine_similarities_volume = cosine_similarity(create_diff_matrix('Volume'), create_diff_matrix('Volume')) * coeff_list[1]
    cosine_similarities_weight = cosine_similarity(create_diff_matrix('Weight'), create_diff_matrix('Weight')) * coeff_list[2]
    cosine_similarities_price = cosine_similarity(create_diff_matrix('PriceOPT'), create_diff_matrix('PriceOPT')) * coeff_list[3]

    cosine_similarities = cosine_similarities_details + cosine_similarities_volume + cosine_similarities_weight + cosine_similarities_price
else:
    print('1')

In [ ]:
def check_conditions(row, similar_row):
    if row['ItemID'] != similar_row['ItemID']:
        pass
    else:
        return False
        
    if row['Seria'] == 'Отсутствует' or similar_row['Seria'] == 'Отсутствует' or row['Seria'] != similar_row['Seria']:
        pass
    else:
        return False

    if row['Main_ID_Razdel'] == similar_row['Main_ID_Razdel']:
        pass
    else:
        return False

In [ ]:
%%time
df_dict = df.to_dict(orient='index')
results = {}
similar_items = []
for idx in df_dict:
    similar_items = []
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    for i in similar_indices:
        if check_conditions(df_dict[idx], df_dict[i]) == False:
            continue
        else:
            similar_items.append((cosine_similarities[idx][i], df_dict[i]['ItemID']))
            break
    if similar_items:
        results[df_dict[idx]['ItemID']] = similar_items
    elif not similar_items:
        continue

CPU times: user 208 ms, sys: 1.53 ms, total: 210 ms
Wall time: 214 ms


In [ ]:
results_df_dict = df.set_index('ItemID').to_dict('index')
def item(id):
    return results_df_dict[id]['FullName']

def recommend_single_item(item_id, num):
    '''
    По определенному коду 1С выдает 
    определенное количество наиболее похожих товаров
    '''
    print("Предсказываем " + str(num) + " товаров похожих на " + results_df_dict[id]['FullName'] + " /// " + item_id)
    print("-------")
    recs = results[item_id][:num]
    for rec in recs:
        print("Рекомендуются: " + item(rec[1]) + " (Точность:" + str(round(rec[0], 4)) + ")" + " /// " + rec[1])
    print('')

def recommend_all(items_id, num_predictions):
    
    '''
    Предсказывает по всем кодам 1С 
    определенное количество наиболее похожих товаров
    по текстовому описанию, цене, объемному весу и категории товара
    '''
    
    results_list = []
    metric_list = []
    columns = ['Код 1С основного товара', 'Наименование основного товара', 'Поставщик основного товара', 
               'Код 1С предсказываемого товара', 'Наименование предсказываемого товара', 'Поставщик предсказываемого товара',
               'Точность предсказания']
    
    for item_id in items_id:
        metric = 0
        count_results = 0
        if item_id in results:
            recs = results[item_id][:num_predictions]
        else:
            continue
        for rec in recs:
            main_item_id = item_id
            main_item_name = results_df_dict[main_item_id]['FullName']
            main_item_vendor = results_df_dict[main_item_id]['Vendor']
            predicted_item_vendor = results_df_dict[rec[1]]['Vendor']
            new_row = [main_item_id, main_item_name, main_item_vendor,
                       rec[1], item(rec[1]), predicted_item_vendor, str(round(rec[0], 4))]
            metric += round(rec[0], 5)
            count_results += 1 
            results_list.append(new_row)
        metric_list.append(metric / count_results)
    print('Среднее значение точности предсказания - ', str(round(sum(metric_list) / len(metric_list), 4)))
    results_dataframe = pd.DataFrame(results_list, columns=columns)
    return results_dataframe

In [ ]:
%%time
results_dataframe = recommend_all(df['ItemID'], 5)
display(results_dataframe.head(10))

#results_dataframe.to_excel('results_dataframe.xlsx')
#files.download('results_dataframe.xlsx')

Среднее значение точности предсказания -  0.7649


,Код 1С основного товара,Наименование основного товара,Поставщик основного товара,Код 1С предсказываемого товара,Наименование предсказываемого товара,Поставщик предсказываемого товара,Точность предсказания
0,К029606,Электроочаг модель FX Black Fobos FX,Белая гвардия,Н065923,Электроочаг модель FX M Black Majestic FX,Белая гвардия,0.5687
1,К015725,Электроочаг Eugene,Real Flame,К015532,Электроочаг MoonBlaze Deluxe,Real Flame,0.6955
2,К029615,Электрокамин очаг Fobos FX Black Venice махаго...,Белая гвардия,Н024920,Электрокамин очаг Fobos FX Black Neapol махагон,Белая гвардия,0.8833
3,Н024920,Электрокамин очаг Fobos FX Black Neapol махагон,Белая гвардия,К029615,Электрокамин очаг Fobos FX Black Venice махаго...,Белая гвардия,0.8833
4,К015532,Электроочаг MoonBlaze Deluxe,Real Flame,Н024439,Электроочаг Majestic Lux Black,Real Flame,0.767
5,К029661,Портал Lumsden,Белая гвардия,К029671,Портал Neapol,Белая гвардия,0.7516
6,К029687,Портал модель dark Cardinal,Белая гвардия,К029694,Портал Boston,Белая гвардия,0.532
7,К029689,Портал Rodos,Белая гвардия,К052437,Портал Adel,Белая гвардия,0.4717
8,К029671,Портал Neapol,Белая гвардия,К029661,Портал Lumsden,Белая гвардия,0.7516
9,К029690,Портал Manchester,Белая гвардия,К029671,Портал Neapol,Белая гвардия,0.5798


CPU times: user 54.8 ms, sys: 1.21 ms, total: 56 ms
Wall time: 68.3 ms


In [ ]:
full_path = '/content/Похожие товары 18112021.csv'
test = pd.read_csv(full_path, encoding='1251', delimiter=';', decimal=',', engine = 'python', index_col=False).astype(str)

In [ ]:
for column in test:
    results_test_list = []
    count_plus = 0
    count_minus = 0  
    for value in test[column]:
        if len(df[df['ItemID'].astype(str) == value]) > 0:
            count_plus += 1
            results_test_list.append(df[df['ItemID'] == value].index[0])
        else:
            count_minus += 1
            results_test_list.append(0)
    test[column] = results_test_list
print(count_plus)
print(count_minus)

90
0


Results_test - сравнение с товарами из блока похожие товары
</br>
Results - модель

In [ ]:
results_test = {}
similar_items_test = []

results = {}
similar_items = []

for idx in test_dict:
    count_i = 0
    similar_items_test = []
    similar_items = []

    similar_indices = cosine_similarities[idx].argsort()[:(len(df)+1)*-1:-1]
    
    for i in similar_indices:
        if i in test_dict[idx]:
            similar_items_test.append((cosine_similarities[idx][i], df_dict[i]['ItemID']))
        
        if check_conditions(df_dict[idx], df_dict[i]) == False:
            continue
        elif count_i < 4:
            similar_items.append((cosine_similarities[idx][i], df_dict[i]['ItemID']))
            count_i += 1

    if similar_items_test:
        results_test[df_dict[idx]['ItemID']] = similar_items_test
    elif not similar_items_test:
        continue

    if similar_items:
        results[df_dict[idx]['ItemID']] = similar_items
    elif not similar_items:
        continue

In [ ]:
metric_list = []
# Вытягиваем строку с предсказаниями
for idx in test_dict:
    # Вытягиваем предсказание
    metric_sublist = []
    for i in range(len(results[df_dict[idx]['ItemID']])):
        row = results[df_dict[idx]['ItemID']][i]
        row_test = results_test[df_dict[idx]['ItemID']][i]
        element = row[0]
        element_test = row_test[0]
        metric_sublist.append(element - element_test)
    metric = sum(metric_sublist) / len(metric_sublist)
    metric_list.append(metric)
print('Разница между средними значениями предсказываемых товаров и существующих похожих товаров -', round(sum(metric_list) / len(metric_list), 4))

Разница между средними значениями предсказываемых товаров и существующих похожих товаров - 0.225
